In [1]:
import os
import talib
import logging
import pandas as pd
from datetime import datetime, timedelta
pd.set_option('display.float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
root_path = "/Users/fang/stock_data/basic_data"
# root_path = "C:\\Users\\Fang\\PycharmProjects\\股票分析\\stock_data\\basic_data"
# root_path = "C:\\Users\\Flora\\Documents\\股票分析\\stock_data\\basic_data"

In [9]:
df_list = []
now = datetime.now()
start_dt = (now-timedelta(days=60)).strftime('%Y-%m-%d')
end_dt = now.strftime('%Y-%m-%d')
bband_slope_change_criteria = 0.5
bband_width_change_criteria = 0.25
prev_bband_width_criteria = 5
for file in sorted(os.listdir(root_path)):
    if '.csv' not in file:
        continue
    logging.info(file.replace('.csv', ''))
    
    # load stock price data
    file_path = root_path + os.sep + file
    try:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python', encoding='utf-8').sort_values('日期')
    except:
        stock_df = pd.read_csv(file_path, index_col=None, header=0, dtype={'股票代號':str}, engine='python').sort_values('日期')
    stock_df.loc[:, '日期'] = pd.to_datetime(stock_df['日期'].str.replace('＊', ''))
    stock_df = stock_df[(stock_df['日期'] >= start_dt) & (stock_df['日期'] <= end_dt)]
    if len(stock_df) < 40:
        continue
    stock_df = stock_df.sort_values('日期')
    stock_df['收盤價'].fillna(method='ffill', inplace=True)
    
    # calculate BBands and some features
    closed = stock_df['收盤價'].values
    upper,middle,lower = talib.BBANDS(closed,20, 2, 2, matype=talib.MA_Type.SMA)
    previous_upper = pd.Series(upper).shift(periods=1).to_numpy()
    previous_lower = pd.Series(lower).shift(periods=1).to_numpy()
    upper_change = pd.Series((upper - previous_upper) / previous_upper*100)
    lower_change = (lower - previous_lower) / previous_lower*100
    bband_width =  pd.Series((upper-lower) / middle*100)
    bband_width_ma = pd.Series(talib.SMA(bband_width, 5))
    
    # add BBands data to stock price dataframe
    stock_df = stock_df.reset_index().drop('index', axis=1)
    stock_df['bband_width'] = bband_width
    stock_df['prev_bband_width'] = stock_df['bband_width'].shift(periods=1)
    stock_df['bband_slope'] = upper_change
    stock_df['prev_bband_slope'] = stock_df['bband_slope'].shift(periods=1)
    stock_df['bband_slope_change'] = abs(stock_df['bband_slope'] - stock_df['prev_bband_slope']) / (abs(stock_df['prev_bband_slope'] + 0.00000001))
    stock_df['bband_width_change'] = (stock_df['bband_width'] - stock_df['prev_bband_width']) / (stock_df['prev_bband_width'] + 0.00000001)
    
    # filter data if meet the alert conditions
    filtered_df = stock_df[(stock_df['bband_slope_change']>bband_slope_change_criteria) 
                           & (stock_df['bband_width_change']>bband_width_change_criteria) 
                           & (stock_df['prev_bband_width']<prev_bband_width_criteria)]
    df_list.append(filtered_df)
alert_df = pd.concat(df_list, axis=0, ignore_index=True, sort=False)

2020-07-28 19:39:06,599 [INFO] 0050
2020-07-28 19:39:06,653 [INFO] 0051
2020-07-28 19:39:06,718 [INFO] 0052
2020-07-28 19:39:06,750 [INFO] 0053
2020-07-28 19:39:06,785 [INFO] 0054
2020-07-28 19:39:06,822 [INFO] 0055
2020-07-28 19:39:06,858 [INFO] 0056
2020-07-28 19:39:06,884 [INFO] 0057
2020-07-28 19:39:06,928 [INFO] 0058
2020-07-28 19:39:06,948 [INFO] 0059
2020-07-28 19:39:06,971 [INFO] 0061
2020-07-28 19:39:07,022 [INFO] 1101
2020-07-28 19:39:07,053 [INFO] 1102
2020-07-28 19:39:07,088 [INFO] 1103
2020-07-28 19:39:07,124 [INFO] 1104
2020-07-28 19:39:07,159 [INFO] 1108
2020-07-28 19:39:07,193 [INFO] 1109
2020-07-28 19:39:07,228 [INFO] 1110
2020-07-28 19:39:07,268 [INFO] 1201
2020-07-28 19:39:07,315 [INFO] 1203
2020-07-28 19:39:07,358 [INFO] 1210
2020-07-28 19:39:07,422 [INFO] 1213
2020-07-28 19:39:07,466 [INFO] 1215
2020-07-28 19:39:07,500 [INFO] 1216
2020-07-28 19:39:07,532 [INFO] 1217
2020-07-28 19:39:07,561 [INFO] 1218
2020-07-28 19:39:07,589 [INFO] 1219
2020-07-28 19:39:07,622 [INF

2020-07-28 19:39:14,530 [INFO] 1795
2020-07-28 19:39:14,556 [INFO] 1796
2020-07-28 19:39:14,583 [INFO] 1799
2020-07-28 19:39:14,609 [INFO] 1802
2020-07-28 19:39:14,636 [INFO] 1805
2020-07-28 19:39:14,669 [INFO] 1806
2020-07-28 19:39:14,698 [INFO] 1808
2020-07-28 19:39:14,731 [INFO] 1809
2020-07-28 19:39:14,764 [INFO] 1810
2020-07-28 19:39:14,796 [INFO] 1813
2020-07-28 19:39:14,828 [INFO] 1815
2020-07-28 19:39:14,862 [INFO] 1817
2020-07-28 19:39:14,895 [INFO] 1902
2020-07-28 19:39:14,913 [INFO] 1903
2020-07-28 19:39:14,947 [INFO] 1904
2020-07-28 19:39:14,985 [INFO] 1905
2020-07-28 19:39:15,020 [INFO] 1906
2020-07-28 19:39:15,062 [INFO] 1907
2020-07-28 19:39:15,094 [INFO] 1909
2020-07-28 19:39:15,126 [INFO] 2002
2020-07-28 19:39:15,161 [INFO] 2006
2020-07-28 19:39:15,187 [INFO] 2007
2020-07-28 19:39:15,214 [INFO] 2008
2020-07-28 19:39:15,244 [INFO] 2009
2020-07-28 19:39:15,273 [INFO] 2010
2020-07-28 19:39:15,297 [INFO] 2012
2020-07-28 19:39:15,321 [INFO] 2013
2020-07-28 19:39:15,353 [INF

2020-07-28 19:39:27,616 [INFO] 2511
2020-07-28 19:39:27,657 [INFO] 2514
2020-07-28 19:39:27,709 [INFO] 2515
2020-07-28 19:39:27,753 [INFO] 2516
2020-07-28 19:39:27,834 [INFO] 2520
2020-07-28 19:39:27,877 [INFO] 2524
2020-07-28 19:39:27,929 [INFO] 2527
2020-07-28 19:39:27,971 [INFO] 2528
2020-07-28 19:39:28,019 [INFO] 2530
2020-07-28 19:39:28,056 [INFO] 2534
2020-07-28 19:39:28,102 [INFO] 2535
2020-07-28 19:39:28,139 [INFO] 2536
2020-07-28 19:39:28,191 [INFO] 2537
2020-07-28 19:39:28,228 [INFO] 2538
2020-07-28 19:39:28,268 [INFO] 2539
2020-07-28 19:39:28,308 [INFO] 2540
2020-07-28 19:39:28,350 [INFO] 2542
2020-07-28 19:39:28,399 [INFO] 2543
2020-07-28 19:39:28,445 [INFO] 2545
2020-07-28 19:39:28,487 [INFO] 2546
2020-07-28 19:39:28,538 [INFO] 2547
2020-07-28 19:39:28,583 [INFO] 2548
2020-07-28 19:39:28,644 [INFO] 2596
2020-07-28 19:39:28,714 [INFO] 2597
2020-07-28 19:39:28,771 [INFO] 2601
2020-07-28 19:39:28,814 [INFO] 2603
2020-07-28 19:39:28,870 [INFO] 2605
2020-07-28 19:39:28,919 [INF

2020-07-28 19:39:35,046 [INFO] 3217
2020-07-28 19:39:35,069 [INFO] 3218
2020-07-28 19:39:35,093 [INFO] 3219
2020-07-28 19:39:35,104 [INFO] 3221
2020-07-28 19:39:35,125 [INFO] 3224
2020-07-28 19:39:35,147 [INFO] 3226
2020-07-28 19:39:35,169 [INFO] 3227
2020-07-28 19:39:35,189 [INFO] 3228
2020-07-28 19:39:35,211 [INFO] 3229
2020-07-28 19:39:35,232 [INFO] 3230
2020-07-28 19:39:35,255 [INFO] 3231
2020-07-28 19:39:35,277 [INFO] 3232
2020-07-28 19:39:35,300 [INFO] 3234
2020-07-28 19:39:35,322 [INFO] 3236
2020-07-28 19:39:35,344 [INFO] 3252
2020-07-28 19:39:35,366 [INFO] 3257
2020-07-28 19:39:35,389 [INFO] 3259
2020-07-28 19:39:35,411 [INFO] 3260
2020-07-28 19:39:35,433 [INFO] 3264
2020-07-28 19:39:35,455 [INFO] 3265
2020-07-28 19:39:35,479 [INFO] 3266
2020-07-28 19:39:35,503 [INFO] 3268
2020-07-28 19:39:35,524 [INFO] 3272
2020-07-28 19:39:35,544 [INFO] 3276
2020-07-28 19:39:35,565 [INFO] 3284
2020-07-28 19:39:35,585 [INFO] 3285
2020-07-28 19:39:35,606 [INFO] 3287
2020-07-28 19:39:35,626 [INF

2020-07-28 19:39:40,675 [INFO] 4137
2020-07-28 19:39:40,699 [INFO] 4138
2020-07-28 19:39:40,719 [INFO] 4139
2020-07-28 19:39:40,741 [INFO] 4141
2020-07-28 19:39:40,763 [INFO] 4142
2020-07-28 19:39:40,910 [INFO] 4144
2020-07-28 19:39:40,934 [INFO] 4147
2020-07-28 19:39:40,956 [INFO] 4148
2020-07-28 19:39:40,980 [INFO] 4152
2020-07-28 19:39:41,006 [INFO] 4153
2020-07-28 19:39:41,030 [INFO] 4154
2020-07-28 19:39:41,054 [INFO] 4155
2020-07-28 19:39:41,078 [INFO] 4157
2020-07-28 19:39:41,105 [INFO] 4160
2020-07-28 19:39:41,128 [INFO] 4161
2020-07-28 19:39:41,162 [INFO] 4162
2020-07-28 19:39:41,201 [INFO] 4163
2020-07-28 19:39:41,235 [INFO] 4164
2020-07-28 19:39:41,256 [INFO] 4167
2020-07-28 19:39:41,279 [INFO] 4168
2020-07-28 19:39:41,304 [INFO] 4171
2020-07-28 19:39:41,326 [INFO] 4173
2020-07-28 19:39:41,348 [INFO] 4174
2020-07-28 19:39:41,372 [INFO] 4175
2020-07-28 19:39:41,394 [INFO] 4180
2020-07-28 19:39:41,405 [INFO] 4183
2020-07-28 19:39:41,431 [INFO] 4188
2020-07-28 19:39:41,481 [INF

2020-07-28 19:39:48,820 [INFO] 5305
2020-07-28 19:39:48,846 [INFO] 5306
2020-07-28 19:39:48,878 [INFO] 5309
2020-07-28 19:39:48,907 [INFO] 5310
2020-07-28 19:39:48,934 [INFO] 5312
2020-07-28 19:39:48,958 [INFO] 5314
2020-07-28 19:39:48,983 [INFO] 5315
2020-07-28 19:39:49,022 [INFO] 5317
2020-07-28 19:39:49,040 [INFO] 5321
2020-07-28 19:39:49,077 [INFO] 5324
2020-07-28 19:39:49,105 [INFO] 5328
2020-07-28 19:39:49,127 [INFO] 5340
2020-07-28 19:39:49,150 [INFO] 5344
2020-07-28 19:39:49,172 [INFO] 5345
2020-07-28 19:39:49,196 [INFO] 5347
2020-07-28 19:39:49,218 [INFO] 5348
2020-07-28 19:39:49,242 [INFO] 5349
2020-07-28 19:39:49,267 [INFO] 5351
2020-07-28 19:39:49,580 [INFO] 5353
2020-07-28 19:39:49,684 [INFO] 5355
2020-07-28 19:39:49,718 [INFO] 5356
2020-07-28 19:39:49,773 [INFO] 5364
2020-07-28 19:39:49,813 [INFO] 5371
2020-07-28 19:39:49,863 [INFO] 5381
2020-07-28 19:39:49,898 [INFO] 5383
2020-07-28 19:39:49,942 [INFO] 5386
2020-07-28 19:39:49,979 [INFO] 5388
2020-07-28 19:39:50,011 [INF

2020-07-28 19:39:56,428 [INFO] 6237
2020-07-28 19:39:56,448 [INFO] 6238
2020-07-28 19:39:56,460 [INFO] 6239
2020-07-28 19:39:56,481 [INFO] 6240
2020-07-28 19:39:56,504 [INFO] 6241
2020-07-28 19:39:56,533 [INFO] 6242
2020-07-28 19:39:56,566 [INFO] 6243
2020-07-28 19:39:56,612 [INFO] 6244
2020-07-28 19:39:56,670 [INFO] 6245
2020-07-28 19:39:56,692 [INFO] 6246
2020-07-28 19:39:56,721 [INFO] 6247
2020-07-28 19:39:56,744 [INFO] 6248
2020-07-28 19:39:56,769 [INFO] 6251
2020-07-28 19:39:56,791 [INFO] 6257
2020-07-28 19:39:56,813 [INFO] 6259
2020-07-28 19:39:56,919 [INFO] 6261
2020-07-28 19:39:56,976 [INFO] 6263
2020-07-28 19:39:57,009 [INFO] 6264
2020-07-28 19:39:57,033 [INFO] 6265
2020-07-28 19:39:57,058 [INFO] 6266
2020-07-28 19:39:57,079 [INFO] 6269
2020-07-28 19:39:57,101 [INFO] 6270
2020-07-28 19:39:57,124 [INFO] 6271
2020-07-28 19:39:57,148 [INFO] 6274
2020-07-28 19:39:57,170 [INFO] 6275
2020-07-28 19:39:57,192 [INFO] 6276
2020-07-28 19:39:57,214 [INFO] 6277
2020-07-28 19:39:57,237 [INF

2020-07-28 19:40:03,156 [INFO] 8121
2020-07-28 19:40:03,189 [INFO] 8131
2020-07-28 19:40:03,225 [INFO] 8147
2020-07-28 19:40:03,249 [INFO] 8150
2020-07-28 19:40:03,272 [INFO] 8155
2020-07-28 19:40:03,294 [INFO] 8163
2020-07-28 19:40:03,467 [INFO] 8171
2020-07-28 19:40:03,533 [INFO] 8176
2020-07-28 19:40:03,591 [INFO] 8182
2020-07-28 19:40:03,637 [INFO] 8183
2020-07-28 19:40:03,665 [INFO] 8201
2020-07-28 19:40:03,687 [INFO] 8210
2020-07-28 19:40:03,708 [INFO] 8213
2020-07-28 19:40:03,733 [INFO] 8215
2020-07-28 19:40:03,754 [INFO] 8222
2020-07-28 19:40:03,775 [INFO] 8234
2020-07-28 19:40:03,797 [INFO] 8240
2020-07-28 19:40:03,818 [INFO] 8249
2020-07-28 19:40:03,840 [INFO] 8255
2020-07-28 19:40:03,862 [INFO] 8261
2020-07-28 19:40:03,883 [INFO] 8271
2020-07-28 19:40:03,904 [INFO] 8277
2020-07-28 19:40:03,926 [INFO] 8279
2020-07-28 19:40:03,948 [INFO] 8284
2020-07-28 19:40:03,966 [INFO] 8287
2020-07-28 19:40:03,977 [INFO] 8289
2020-07-28 19:40:03,998 [INFO] 8291
2020-07-28 19:40:04,018 [INF

In [10]:
alert_df[alert_df['日期']=='2020-07-28'].sort_values(['日期', '成交金額'])

,股票代號,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,漲跌幅,成交筆數,bband_width,prev_bband_width,bband_slope,prev_bband_slope,bband_slope_change,bband_width_change
222,6624,2020-07-28,11000.00,396700.00,33.75,36.50,33.75,36.50,-1.00,-2.67,9.00,7.24,4.63,0.98,0.02,51.97,0.56
237,8929,2020-07-28,46422.00,1136033.00,24.75,24.75,24.05,24.40,-0.45,-1.81,41.00,6.06,4.83,0.35,-0.27,2.32,0.25
119,2916,2020-07-28,105000.00,1657500.00,15.80,15.90,15.60,15.60,-0.25,-1.58,32.00,2.35,1.34,0.39,0.03,11.24,0.75
134,3531,2020-07-28,88000.00,1895850.00,21.40,21.80,21.20,21.70,-1.00,-4.41,67.00,8.93,4.16,1.96,0.04,50.14,1.15
203,6195,2020-07-28,76257.00,2449761.00,32.30,32.35,32.00,32.00,-1.00,-3.03,54.00,6.86,2.79,1.65,0.00,165217062.71,1.46
211,6417,2020-07-28,46150.00,2720024.00,56.90,60.00,56.90,59.00,2.10,3.69,42.00,9.10,4.89,2.53,-26.27,1.10,0.86
243,9927,2020-07-28,111200.00,3310338.00,29.90,29.90,29.70,29.70,-0.20,-0.67,80.00,3.07,2.44,0.21,0.13,0.63,0.26
85,2506,2020-07-28,371712.00,3391357.00,9.49,9.49,8.96,9.13,-0.42,-4.40,146.00,8.23,4.18,1.62,0.44,2.69,0.97
156,4754,2020-07-28,82000.00,3661600.00,45.25,45.25,44.10,44.35,-1.00,-2.21,62.00,15.00,2.97,4.96,0.03,167.55,4.04
132,3388,2020-07-28,58845.00,3695372.00,63.00,63.00,62.60,62.60,-0.50,-0.79,53.00,2.77,2.17,0.23,0.03,7.40,0.28
